Ron steinmetz
315578575

In [371]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

Data Mining

In [355]:
driver = webdriver.Chrome()

try:
    all_frames = []
    for i in range(1, 3):
        url = f'https://www.ad.co.il/car?sp261=13915&pageindex={i}'
        driver.get(url)

        try:
            cards_wrap = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'cards-wrap'))
            )

            card_blocks = cards_wrap.find_elements(By.CLASS_NAME, 'card-block')
            
            for card_block in card_blocks:
                try:
                    driver.execute_script("arguments[0].scrollIntoView();", card_block)
                    WebDriverWait(driver, 5).until(EC.element_to_be_clickable(card_block)).click()
                    
                    try:
                        title = WebDriverWait(driver, 1).until(
                            EC.presence_of_element_located((By.XPATH, '//*[@id="product-container"]/div/div[1]/div/div[1]/h2[1]'))
                        ).text
                        
                    except TimeoutException:
                        title = "Title not found"
                    except NoSuchElementException:
                        title = "Title not found"
                    
                    try:
                        price = WebDriverWait(driver, 2).until(
                            EC.presence_of_element_located((By.XPATH, '//*[@id="product-container"]/div/div[1]/div/div[1]/h2[2]'))
                        ).text
                        price = price.replace("₪", "").strip()
                    except TimeoutException:
                        price = ""
                    except NoSuchElementException:
                        price = ""
                    
                    try:
                        main = WebDriverWait(driver, 1).until(
                            EC.presence_of_element_located((By.CLASS_NAME, 'table.table-sm'))
                        ).text
                    except TimeoutException:
                        main = ""
                    except NoSuchElementException:
                        main = ""
                    
                    try:
                        text = WebDriverWait(driver, 1).until(
                            EC.presence_of_element_located((By.CLASS_NAME, 'text-word-break'))
                        ).text
                    except TimeoutException:
                        text = ""
                    except NoSuchElementException:
                        text = ""
                    
                    try:
                        date = WebDriverWait(driver, 1).until(
                            EC.presence_of_element_located((By.CLASS_NAME, 'd-flex.flex-row.align-items-center.justify-content-center.flex-wrap'))
                        ).text.split('\n')
                    except TimeoutException:
                        date = "Date not found"
                    except NoSuchElementException:
                        date = "Date not found"
                    
                    html = card_block.get_attribute('outerHTML')
                    soup = BeautifulSoup(html, 'html.parser')
                    number_of_images = soup.find('div', class_='card-block')['data-images']

                    frame = [title, price, main, text, date[0], date[1],number_of_images]
                    all_frames.append(frame)
                    
                    try:
                        close_button = WebDriverWait(driver, 1).until(
                            EC.element_to_be_clickable((By.XPATH, '//*[@id="pop-modal"]/div/div/div[1]/div/button'))
                        )
                        close_button.click()
                    except TimeoutException:
                        print("Close button timeout")
                    except NoSuchElementException:
                        print("Close button not found")

                except ElementClickInterceptedException:
                    print(f"ElementClickInterceptedException encountered. Retrying...")
                    driver.execute_script("arguments[0].click();", card_block)

                except (TimeoutException, NoSuchElementException) as e:
                    print(f"Error processing card: {e}")
                    continue

        except TimeoutException as e:
            print(f"Error loading page {i}: {e}")
            continue

finally:
    driver.quit()

df = pd.DataFrame(all_frames)


The arrangement of the data

In [356]:
df[['manufactor', 'model']] = df[0].str.split(" ", n=1, expand=True)
df['Price'] = df[1].str.replace(',', '')
temp = df[2].str.split('\n',expand=True)

In [357]:
df['Year'] = temp.apply(lambda row: next((str(value).replace('שנה', '').strip()  for value in row if 'שנה' in str(value)), pd.NA),axis = 1)
df['Hand'] = temp.apply(lambda row: next((str(value).replace('יד', '').strip()  for value in row if 'יד' in str(value)), pd.NA),axis = 1)
df['Gear'] = temp.apply(lambda row: next((str(value).replace('ת. הילוכים', '').strip()  for value in row if 'ת. הילוכים' in str(value)), pd.NA),axis = 1)
df['Engine_capacity'] = temp.apply(lambda row: next((str(value).replace('נפח', '').strip()  for value in row if 'נפח' in str(value)), pd.NA),axis = 1)
df['Engine_type'] = temp.apply(lambda row: next((str(value).replace('סוג מנוע', '').strip()  for value in row if 'סוג מנוע' in str(value)), pd.NA),axis = 1)
df['Prev_ownership'] = temp.apply(lambda row: next((str(value).replace('בעלות קודמת', '').strip()  for value in row if 'בעלות קודמת' in str(value)), pd.NA),axis = 1)
df['Curr_ownership'] = temp.apply(lambda row: next((str(value).replace('בעלות נוכחית', '').strip()  for value in row if 'בעלות נוכחית' in str(value)), pd.NA),axis = 1)
df['Area'] = temp.apply(lambda row: next((str(value).replace('אזור', '').strip()  for value in row if 'אזור' in str(value)), pd.NA),axis = 1)
df['City'] = temp.apply(lambda row: next((str(value).replace('עיר', '').strip()  for value in row if 'עיר' in str(value)), pd.NA),axis = 1)
df['Color'] = temp.apply(lambda row: next((str(value).replace('צבע', '').strip()  for value in row if 'צבע' in str(value)), pd.NA),axis = 1)
df['Km'] = temp.apply(lambda row: next((str(value).replace('ק"מ', '').strip()  for value in row if 'ק"מ' in str(value)), pd.NA),axis = 1)
df['Test'] = temp.apply(lambda row: next((str(value).replace('טסט עד', '').strip()  for value in row if 'טסט עד' in str(value)), pd.NA),axis = 1)

In [358]:
df = df.rename(columns= {3:'Description',4:'Cre_date',5:'Repub_date',6:'Pic_num'})
df['Km'] = df['Km'].str.replace(',', '')
df['Engine_capacity'] = df['Engine_capacity'].str.replace(',', '')
df['Repub_date'] = df['Repub_date'].str.replace('תאריך הקפצה אחרון:', '').str.strip()
df['Cre_date'] = df['Cre_date'].str.replace('תאריך יצירה:','')
df = df.drop(columns=[0,1,2])
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

In [359]:
df = df[['manufactor','Year','model','Hand','Gear','Engine_capacity','Engine_type','Prev_ownership','Curr_ownership','Area','City','Price','Pic_num','Cre_date','Repub_date','Description','Color','Km','Test']]

Converting the data to types

In [360]:
df['Test'] = pd.to_datetime(df['Test'])
df['Test'] = (df['Test'] - current_date).dt.days

In [368]:
df['Year'] = df['Year'].astype(int)
df['Hand'] = df['Hand'].astype(int)
df['Gear'] = df['Gear'].astype('category')
df['Engine_capacity'] = df['Engine_capacity'].astype(int)
df['Engine_type'] = df['Engine_type'].astype('category')
df['Prev_ownership'] = df['Prev_ownership'].astype('category')
df['Curr_ownership'] = df['Curr_ownership'].astype('category')
df['Area'] = df['Area'].astype('string')
df['City'] = df['City'].astype('string')
df['Price'] = df['Price'].astype('float64')
df['Pic_num'] = df['Pic_num'].astype(int)
df['Cre_date'] = pd.to_datetime(df['Cre_date'])
df['Repub_date'] = pd.to_datetime(df['Cre_date'])
df['Description'] = df['Description'].astype('string')
df['Color'] = df['Color'].astype('string')
df['Km'] = df['Km'].astype('Int64')
df['Test'] = df['Test'].astype('Int64')
df.dtypes

manufactor                 object
Year                        int32
model                      object
Hand                        int32
Gear                     category
Engine_capacity             int32
Engine_type              category
Prev_ownership           category
Curr_ownership           category
Area                       string
City                       string
Price                     float64
Pic_num                     int32
Cre_date           datetime64[ns]
Repub_date         datetime64[ns]
Description                string
Color                      string
Km                          Int64
Test                        Int64
dtype: object

Viewing and transferring the data to Excel

In [367]:
pd.concat([df.head(5), df.tail(5)])

,manufactor,Year,model,Hand,Gear,Engine_capacity,Engine_type,Prev_ownership,Curr_ownership,Area,City,Price,Pic_num,Cre_date,Repub_date,Description,Color,Km,Test
0,אופל,2016,אסטרה,2,אוטומטית,1400,בנזין,פרטית,פרטית,באר שבע והסביבה,באר שבע,24000.0,4,2024-03-16,2024-03-16,"רכב שמור מאוד,נהגת יחידה,מוטימדיה,בקרת שיוט,רכ...",אפור מטאלי,141000,<NA>
1,אופל,2014,אסטרה,3,אוטומטית,1400,בנזין,פרטית,פרטית,בית שמש והסביבה,בית שמש,18000.0,4,2024-12-05,2024-12-05,לאוהבי המציאות!!! אופל אסטרה ברלינה אוטו שמור ...,חום מטאלי,75000,79
2,אופל,2015,קורסה,2,אוטומטית,1400,בנזין,ליסינג,פרטית,<NA>,עין מאהל,15000.0,5,2024-05-22,2024-05-22,אוטו שמור מאוד וחיסכוני. עבר טיפול חזק לפני שב...,אפור מטאלי,175000,<NA>
3,אופל,2016,אסטרה,2,אוטומטית,1400,דיזל,השכרה,פרטית,ירושלים והסביבה,ירושלים,20000.0,2,2024-03-24,2024-03-24,"רכב נקי באיכות גרמנית! הכול עובד כמו חדש, טסט ...",אפור מטאלי,231000,291
4,אופל,2013,אסטרה,2,אוטומטית,1400,בנזין,ליסינג,פרטית,נס ציונה - רחובות,מזכרת בתיה,22000.0,5,2023-12-12,2023-12-12,רכב במצב מצויין שמור מאוד לאחר טיפול. מער' מול...,אפור מטאלי,145000,109
85,אופל,2015,אינסיגניה,1,טיפטרוניק,2000,בנזין,NaN,NaN,רמת השרון - הרצליה,הרצליה,40000.0,0,2022-07-27,2022-07-27,רכב שמור מאוד,<NA>,<NA>,<NA>
86,אופל,2014,אסטרה,1,אוטומטית,1400,בנזין,NaN,NaN,אשדוד - אשקלון,אשקלון,34000.0,0,2022-07-21,2022-07-21,נהג יחיד מוחזק מצויין &lt;br/&gt;&lt;br/&gt;פי...,<NA>,122000,<NA>
87,אופל,2013,אסטרה,2,אוטומטית,1400,בנזין,NaN,NaN,חולון - בת ים,אזור,28000.0,0,2022-07-21,2022-07-21,"יד ראשונה ליסינג, טסט עד 26/10/2022",<NA>,184000,<NA>
88,אופל,2015,מריבה,3,טיפטרוניק,1400,בנזין,NaN,NaN,באר שבע והסביבה,באר שבע,26000.0,0,2022-07-16,2022-07-16,,<NA>,128000,<NA>
89,אופל,2014,אסטרה,2,אוטומטית,1600,בנזין,השכרה,פרטית,אשדוד - אשקלון,גן יבנה,23900.0,0,2021-11-15,2021-11-15,רכב מעולה.מומלץ ביותר.בגז' ענק.מהיר ואמין.,אפור מטאלי,162000,<NA>


In [363]:
df.to_csv('Data mining.csv',index=False ,encoding = 'utf-8-sig')